In [1]:
!pip install google-cloud-storage
!pip install pandas

from google.colab import auth
from google.cloud import storage
import pandas as pd

auth.authenticate_user()
client = storage.Client()

# bucket = client.get_bucket('hons-project-datasets')
# blob = bucket.blob('original/train.json')
# blob.download_to_filename('train.json')

In [2]:
bucket = client.get_bucket('hons-project-datasets')
blob = bucket.blob('new/train_skip_no_refs.json')
blob.download_to_filename('train.json')

In [3]:
df = pd.read_json('train.json')
df.iloc[4].table_text

['Only 13% of young women in Mali are in the highest tercile for empowerment compared with 81% of young women in the Philippines.',
 'Nigeria is in the middle, with about 1/3 of women in each empowerment group (Figure 1).']

In [4]:
def join_strings(text_list):
    return ' '.join(text_list)

df['table_text'] = df['table_text'].apply(join_strings)

In [30]:
df.iloc[513].table_text

'Mlingano wa viwango vya utafiti wa KDHS wa 2008-09 na utafiti wa KDHS wa mwaka wa 2003 unaonyesha kupungua kwa idadi ya vifo vya watu wazima kwa wanawake na wanaume, lakini mitindo inatofautiana kiasi Viwango vya vifo vya wanawake wazima kutoka kwa data ya 2008-09 ni vya chini kwa umri wote, isipokuwa kuanzia miaka 35 kuendelea, ambapo viwango vinakaribia kuwa sawa na vile vya utafiti wa mwaka wa 2003. Idadi ya vifo vya wanaume wazima ipo chini kwa vikundi zaidi vya wanarika, isipokuwa vikundi vya miaka 15-19 na 45-49. Muhtasari wa kipimo cha idadi ya vifo kwa umri wa kati ya 15-49 unaonyesha kupungua kwa karibu asilimia 12 katika idadi ya vifo vya wanawake lakini asilimia 3 pekee kwa kupungua kwa idadi ya vifo vya wanaume kutoka viwango vya utafiti wa KDHS mwaka wa 2003.'

In [6]:
df = df.drop('target', axis=1)

KeyError: ignored

In [8]:
df.head()

example_id                                                  AB20-en-1
title               Youth Empowerment among Young Women age 15-29 ...
unit_of_measure                         percentage in pooled terciles
chart_type                                       Horizontal Bar Chart
was_translated                                                  False
table_data          [["", "High Empowerment", "Medium Empowerment"...
table_text          Only 13% of young women in Mali are in the hig...
linearized_input    Youth Empowerment among Young Women age 15-29 ...
Name: 4, dtype: object

In [32]:
csv_file_path = 'test_joined_refs.csv'
df.to_csv(csv_file_path, sep='\t', index=False)

In [12]:
import pandas as pd

bucket = client.get_bucket('hons-project-datasets')
blob = bucket.blob('new/test_with_targets.json')
blob.download_to_filename('test.json')

# Read the JSON file into a DataFrame
json_file_path = 'test.json'
df = pd.read_json(json_file_path)

# List of columns to drop
columns_to_drop = ['example_id', 'title', 'unit_of_measure', 'chart_type', 'was_translated', 'table_data', 'table_text']

# Drop the specified columns
df = df.drop(columns=columns_to_drop)

# Rename columns if needed
# df = df.rename(columns={'table_text': 'target'})

# df = df[['linearized_input', 'target']]

# Write the remaining columns to a CSV file
csv_file_path = 'test.csv'
df.to_csv(csv_file_path, sep='\t', index=False)

In [15]:
df = pd.read_csv('train.csv', sep='\t')
df.head(10)

,linearized_input,target
0,تمكين الشباب بين الشابات في الفئة العمرية من 1...,تتمتع 13% من الشابات في مالي بأعلى مستويات الت...
1,تمكين الشباب بين الشابات في الفئة العمرية من 1...,تحتل نيجيريا منتصف المجموعات، حيث تتضمن كل مجم...
2,العدد المثالي للأطفال وفقًا لمستوى التمكين - ن...,في نيجيريا، ترغب الشابات ذوات المستوى المنخفض ...
3,العدد المثالي للأطفال وتمكين الشابات: تحليل مت...,تُظهر النماذج متعددة المتغيرات أنه في 5 بلدان،...
4,العدد المثالي للأطفال وتمكين الشابات: تحليل مت...,حتى عند التحكم في الخصائص الخلفية، ترغب الشابا...
5,العدد المثالي للأطفال وتمكين الشابات: تحليل مت...,تنطبق هذه العلاقة فحسب في مالاوي على الشابات ذ...
6,الاستخدام الحالي لوسائل منع الحمل والنية لاستخ...,في نيجيريا، تكون الشابات ذوات التمكين العالي أ...
7,Youth Empowerment among Young Women age 15-29 ...,Only 13% of young women in Mali are in the hig...
8,Youth Empowerment among Young Women age 15-29 ...,"Nigeria is in the middle, with about 1/3 of wo..."
9,Ideal Number of Children by Empowerment - Nige...,"In Nigeria, young women with low empowerment w..."


In [ ]:
df = pd.read_json('test.json')

if 'table_text' in df.columns:
    df.drop(columns=['table_text'], inplace=True)

# Rename the 'target' column to 'table_text'
if 'target' in df.columns:
    df.rename(columns={'target': 'table_text'}, inplace=True)

df.to_json('output.jsonl', orient='records', lines=True)

In [ ]:
df = pd.read_json('test.json')

# Filter the rows where 'table_text' column is empty list i.e. no references
no_refs_df = df[df['table_text'].apply(lambda x: len(x) > 0)]

no_refs_df

len(no_refs_df[no_refs_df['table_text'].apply(lambda x: len(x) == 0)])

0

In [ ]:
len(df)

6962

In [ ]:
df = pd.read_json('train.json')

# Filter the rows where 'table_text' column is empty list i.e. no references
empty_df = df[df['table_text'].apply(lambda x: len(x) == 0)]

len(empty_df)

2180

In [ ]:
blob = bucket.blob('new/train_cleaned.json')
blob.download_to_filename('train_cleaned.json')

In [ ]:
train_cleaned_df = pd.read_json('train_cleaned.json')

In [ ]:
len(train_cleaned_df)

6962

In [ ]:
# Filter out the rows where 'table_text' column is empty list i.e. no references
train_cleaned_skip_no_refs_df = train_cleaned_df[train_cleaned_df['table_text'].apply(lambda x: len(x) > 0)]

0

In [ ]:
# Expand
train_cleaned_skip_no_refs_exploded_df = train_cleaned_skip_no_refs_df.explode('table_text')
train_cleaned_skip_no_refs_exploded_df.reset_index(drop=True, inplace=True)

In [ ]:
print(len(train_cleaned_skip_no_refs_df))
len(train_cleaned_skip_no_refs_exploded_df)

4782


7060

In [ ]:
blob = bucket.blob('original/.json')
blob.download_to_filename('test.json')
test_df = pd.read_json('test.json')

In [ ]:
# Define a function to extract the first element from the list
def get_first_element(lst):
    if isinstance(lst, list) and len(lst) > 0:
        return lst[0]
    else:
        return None  # Handle empty lists or non-lists

# Apply the function to create a new column
test_df['target'] = test_df['table_text'].apply(get_first_element)

In [ ]:
train_cleaned_skip_no_refs_exploded_df.to_json('new/train_skip_no_refs_exploded.json', orient='records')

In [ ]:
test_df.head()

,example_id,title,unit_of_measure,chart_type,was_translated,table_data,table_text,linearized_input,target
0,AS75-ar-2,متوسط الأعمار عند ممارسة أول علاقة جنسية، والز...,الأعمار (بالسنوات),Other,True,"[["""", ""\u0623\u0648\u0644 \u0639\u0644\u0627\u...",[هناك عدة فروقات شائعة بين المناطق الريفية وال...,متوسط الأعمار عند ممارسة أول علاقة جنسية، والز...,هناك عدة فروقات شائعة بين المناطق الريفية والح...
1,AS75-ar-6,مسارات تكوين الأسرة بين الرجال الذين تتراوح أع...,,Other,True,"[["""", ""\u062a\u0648\u0642\u064a\u062a \u0645\u...",[في مخطط سانكي الأول لبنين (الشكل رقم 6)، بلغت...,مسارات تكوين الأسرة بين الرجال الذين تتراوح أع...,في مخطط سانكي الأول لبنين (الشكل رقم 6)، بلغت ...
2,AS75-ar-7,مسارات تكوين الأسرة بين الرجال الذين تتراوح أع...,,Other,True,"[["""", ""\u062a\u0648\u0642\u064a\u062a \u0645\u...",[في مخطط سانكي الأول لبنين (الشكل رقم 6)، بلغت...,مسارات تكوين الأسرة بين الرجال الذين تتراوح أع...,في مخطط سانكي الأول لبنين (الشكل رقم 6)، بلغت ...
3,AS75-en-2,"Median ages at first sex, first marriage, and ...",Age (years),Other,False,"[["""", ""First sex"", ""First marriage"", ""Birth of...",[A number of the differences between rural and...,"Median ages at first sex, first marriage, and ...",A number of the differences between rural and ...
4,AS75-en-6,Family formation trajectories among men age 30...,,Other,False,"[["""", ""Timing of first sex"", ""Timing of first ...",[In the first Sankey diagram for Benin (Figure...,Family formation trajectories among men age 30...,In the first Sankey diagram for Benin (Figure ...


In [ ]:
# Convert the DataFrame to a JSON string
json_string = test_df.to_json(orient='records')

# Get the bucket and file objects
bucket = client.get_bucket('hons-project-datasets')
blob = bucket.blob('new/test_with_targets.json')

# Upload the JSON string to the bucket
blob.upload_from_string(json_string, content_type='application/json')

In [ ]:
total_element_count = sum(len(lst) for lst in df['table_text'])
total_element_count

7414

In [ ]:
total_element_count = sum(len(lst) for lst in train_cleaned_df['table_text'])
total_element_count

7060

In [ ]:
train_cleaned_df['table_text'].apply(lambda x: len(x) == 0).sum()

2180

In [ ]:
df = df[df['table_text'].apply(lambda x: len(x) > 0)]
df = df.explode('table_text')
df.reset_index(drop=True, inplace=True)
len(df)

7414

In [ ]:
# Convert the DataFrame to a JSON string
json_string = no_refs_df.to_json(orient='records')

# Get the bucket and file objects
bucket = client.get_bucket('hons-project-datasets')
blob = bucket.blob('modified/train_skip_no_refs.json')

# Upload the JSON string to the bucket
# blob.upload_from_string(json_string, content_type='application/json')

In [ ]:
# Filter the rows where 'table_text' column has 1 element or less
multiple_refs_df = df[df['table_text'].apply(lambda x: len(x) > 1)]

multiple_refs_df.head()

,example_id,title,unit_of_measure,chart_type,was_translated,table_data,table_text,linearized_input
0,AB20-ar-1,تمكين الشباب بين الشابات في الفئة العمرية من 1...,النسبة المئوية موضحة في ثلاثة أجزاء مُجمعة,Horizontal Bar Chart,True,"[["""", ""\u0645\u0633\u062a\u0648\u0649 \u0639\u...",[تتمتع 13% من الشابات في مالي بأعلى مستويات ال...,تمكين الشباب بين الشابات في الفئة العمرية من 1...
2,AB20-ar-3,العدد المثالي للأطفال وتمكين الشابات: تحليل مت...,معاملات تمكين الشابات للعدد المثالي للأطفال من...,Other,True,"[["""", ""\u0645\u0633\u062a\u0648\u0649 \u0645\u...",[تُظهر النماذج متعددة المتغيرات أنه في 5 بلدان...,العدد المثالي للأطفال وتمكين الشابات: تحليل مت...
4,AB20-en-1,Youth Empowerment among Young Women age 15-29 ...,percentage in pooled terciles,Horizontal Bar Chart,False,"[["""", ""High Empowerment"", ""Medium Empowerment""...",[Only 13% of young women in Mali are in the hi...,Youth Empowerment among Young Women age 15-29 ...
6,AB20-en-3,Ideal Number of Children and Young Women’s Emp...,Youth empowerment coecients for ideal number ...,Other,False,"[["""", ""Medium Empowerment"", ""High Empowerment""...",[Mutlivariable models show that in 5 countries...,Ideal Number of Children and Young Women’s Emp...
8,AB20-fr-1,Niveau d'autonomisation des jeunes femmes de 1...,pourcentage selon des groupes de terciles,Horizontal Bar Chart,True,"[["""", ""Niveau d'autonomisation \u00e9lev\u00e9...",[Seules 13 % des jeunes femmes au Mali se situ...,Niveau d'autonomisation des jeunes femmes de 1...


In [ ]:
blob = bucket.blob('modified/train_skip_no_refs.json')
blob.download_to_filename('train_skip_no_refs.json')

train_skip_no_refs_df = pd.read_json('train_skip_no_refs.json')
train_skip_no_refs_df.head()

,example_id,title,unit_of_measure,chart_type,was_translated,table_data,table_text,linearized_input
0,AB20-ar-1,تمكين الشباب بين الشابات في الفئة العمرية من 1...,النسبة المئوية موضحة في ثلاثة أجزاء مُجمعة,Horizontal Bar Chart,True,"[["""", ""\u0645\u0633\u062a\u0648\u0649 \u0639\u...",[تتمتع 13% من الشابات في مالي بأعلى مستويات ال...,تمكين الشباب بين الشابات في الفئة العمرية من 1...
1,AB20-ar-2,العدد المثالي للأطفال وفقًا لمستوى التمكين - ن...,العدد بين الشابات في الفئة العمرية من 15 إلى 2...,Line Chart,True,"[[""\u0645\u0633\u062a\u0648\u0649 \u0645\u0646...",[في نيجيريا، ترغب الشابات ذوات المستوى المنخفض...,العدد المثالي للأطفال وفقًا لمستوى التمكين - ن...
2,AB20-ar-3,العدد المثالي للأطفال وتمكين الشابات: تحليل مت...,معاملات تمكين الشابات للعدد المثالي للأطفال من...,Other,True,"[["""", ""\u0645\u0633\u062a\u0648\u0649 \u0645\u...",[تُظهر النماذج متعددة المتغيرات أنه في 5 بلدان...,العدد المثالي للأطفال وتمكين الشابات: تحليل مت...
3,AB20-ar-4,الاستخدام الحالي لوسائل منع الحمل والنية لاستخ...,النسبة المئوية بين الشابات في الفئة العمرية من...,Line Chart,True,"[["""", ""\u0627\u0644\u0646\u064a\u0629 \u0641\u...",[في نيجيريا، تكون الشابات ذوات التمكين العالي ...,الاستخدام الحالي لوسائل منع الحمل والنية لاستخ...
4,AB20-en-1,Youth Empowerment among Young Women age 15-29 ...,percentage in pooled terciles,Horizontal Bar Chart,False,"[["""", ""High Empowerment"", ""Medium Empowerment""...",[Only 13% of young women in Mali are in the hi...,Youth Empowerment among Young Women age 15-29 ...


In [ ]:
len(train_skip_no_refs_df)

4782

In [ ]:
blob = bucket.blob('original/test.json')
blob.download_to_filename('test.json')

test_df = pd.read_json('test.json')
test_df.head()

,example_id,title,unit_of_measure,chart_type,was_translated,table_data,table_text,linearized_input
0,AS75-ar-2,متوسط الأعمار عند ممارسة أول علاقة جنسية، والز...,الأعمار (بالسنوات),Other,True,"[["""", ""\u0623\u0648\u0644 \u0639\u0644\u0627\u...",[هناك عدة فروقات شائعة بين المناطق الريفية وال...,متوسط الأعمار عند ممارسة أول علاقة جنسية، والز...
1,AS75-ar-6,مسارات تكوين الأسرة بين الرجال الذين تتراوح أع...,,Other,True,"[["""", ""\u062a\u0648\u0642\u064a\u062a \u0645\u...",[في مخطط سانكي الأول لبنين (الشكل رقم 6)، بلغت...,مسارات تكوين الأسرة بين الرجال الذين تتراوح أع...
2,AS75-ar-7,مسارات تكوين الأسرة بين الرجال الذين تتراوح أع...,,Other,True,"[["""", ""\u062a\u0648\u0642\u064a\u062a \u0645\u...",[في مخطط سانكي الأول لبنين (الشكل رقم 6)، بلغت...,مسارات تكوين الأسرة بين الرجال الذين تتراوح أع...
3,AS75-en-2,"Median ages at first sex, first marriage, and ...",Age (years),Other,False,"[["""", ""First sex"", ""First marriage"", ""Birth of...",[A number of the differences between rural and...,"Median ages at first sex, first marriage, and ..."
4,AS75-en-6,Family formation trajectories among men age 30...,,Other,False,"[["""", ""Timing of first sex"", ""Timing of first ...",[In the first Sankey diagram for Benin (Figure...,Family formation trajectories among men age 30...


In [ ]:
# Filter the rows where 'table_text' column is empty list i.e. no references
test_no_refs_df = test_df[test_df['table_text'].apply(lambda x: len(x) > 1)]

print(len(test_no_refs_df))
len(test_df)

763


763

In [ ]:
english_df = no_refs_df[no_refs_df['example_id'].str.contains("-en-")]

# Convert the DataFrame to a JSON string
json_string = english_df.to_json(orient='records')

# Get the bucket and file objects
bucket = client.get_bucket('hons-project-datasets')
blob = bucket.blob('modified/train_skip_no_refs_english.json')

# Upload the JSON string to the bucket
# blob.upload_from_string(json_string, content_type='application/json')

In [ ]:
english_test_df = test_df[test_df['example_id'].str.contains("-en-")]

# Convert the DataFrame to a JSON string
json_string = english_test_df.to_json(orient='records')

# Get the bucket and file objects
bucket = client.get_bucket('hons-project-datasets')
blob = bucket.blob('modified/test_english.json')

# Upload the JSON string to the bucket
# blob.upload_from_string(json_string, content_type='application/json')

In [ ]:
english_test_df

,example_id,title,unit_of_measure,chart_type,was_translated,table_data,table_text,linearized_input
3,AS75-en-2,"Median ages at first sex, first marriage, and ...",Age (years),Other,False,"[["""", ""First sex"", ""First marriage"", ""Birth of...",[A number of the differences between rural and...,"Median ages at first sex, first marriage, and ..."
4,AS75-en-6,Family formation trajectories among men age 30...,,Other,False,"[["""", ""Timing of first sex"", ""Timing of first ...",[In the first Sankey diagram for Benin (Figure...,Family formation trajectories among men age 30...
5,AS75-en-7,Family formation trajectories among men age 30...,,Other,False,"[["""", ""Timing of first sex"", ""To Earlier"", ""To...",[In the first Sankey diagram for Benin (Figure...,Family formation trajectories among men age 30...
25,AS77-en-1,Youth empowerment (pooled terciles) among wome...,,Horizontal bar chart,False,"[["""", ""High"", ""Medium"", ""Low""], [""Mali"", 13.2,...",[A mere 13% of young women are in the high emp...,Youth empowerment (pooled terciles) among wome...
34,DM52-en-4,Ownership of House and Land,Percent of women and men age 15-49 who:,Vertical bar chart,False,"[["""", ""Women"", ""Men""], [""Own a home alone or j...","[Only 18% of women own a house, either alone o...",Ownership of House and Land | Percent of women...
...,...,...,...,...,...,...,...,...
696,SR280-en-8,Percent of children vaccinated against better ...,Percent of children vaccinated,Horizontal bar chart,False,"[[""CHILDREN WHO RECEIVED"", ""Percent of childre...",[Children who are immunised have better chance...,Percent of children vaccinated against better ...
735,SR65-en-5,Current Use of Contraception among Women Curre...,Percent,Horizontal bar chart,False,"[["""", """", ""Modern"", ""Traditional"", """"], ["""", ""...",[There are considerabble differences in contra...,Current Use of Contraception among Women Curre...
736,SR65-en-6,Total Demand for family Planning (Unmet Need a...,Percent,Horizontal bar chart,False,"[["""", """", ""Unmet Need"", ""Current Use"", """"], [""...",[The total demand for family planning among no...,Total Demand for family Planning (Unmet Need a...
749,TR8-en-16,Percentage of currently married women age 15-4...,,Vertical bar chart,False,"[["""", ""Unmet need for spacing"", ""Unmet need fo...","[However, the percent of currently married wom...",Percentage of currently married women age 15-4...


In [ ]:
no_refs_df['table_text'].duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
6957    False
6958    False
6959    False
6960    False
6961    False
Name: table_text, Length: 4782, dtype: bool

In [ ]:
# Count the number of rows where any of the elements in 'table_text' contain "START OF TEXT"
print(len(no_refs_cleaned_df))
no_refs_cleaned_df = no_refs_df[no_refs_df['table_text'].apply(lambda x: not (any("START OF TEXT" in item for item in x)))]
print(len(no_refs_cleaned_df))

no_refs_cleaned_df = no_refs_cleaned_df[~no_refs_cleaned_df['example_id'].str.contains("OF29")]
print(len(no_refs_cleaned_df))

# Copy the original DataFrame to a new DataFrame
expanded_df = no_refs_cleaned_df.copy()

print(len(expanded_df))

# Explode the 'table_text' column to create new rows for each element in the lists
expanded_df = expanded_df.explode('table_text')

print(len(expanded_df))

expanded_df

NameError: ignored

In [ ]:
dm51_df = no_refs_df[no_refs_df['example_id'].str.contains("DM51-en-4")]
dm51_df

,example_id,title,unit_of_measure,chart_type,was_translated,table_data,table_text,linearized_input
425,DM51-en-4,Trends in Vaccination Coverage,Percent of children age 12-23 months vaccinated,Vertical Bar Chart,False,"[["""", ""2003 NDHS"", ""2008 NDHS"", ""2013 NDHS""], ...",[Vaccination coverage has gradually increased ...,Trends in Vaccination Coverage | Percent of ch...


In [ ]:
# Filter the rows where 'table_text' column is empty list i.e. no references
how_many_df = expanded_df[expanded_df['table_text'].apply(lambda x: len(x) == 0)]

how_many_df

,example_id,title,unit_of_measure,chart_type,was_translated,table_data,table_text,linearized_input


In [ ]:
# Convert the DataFrame to a JSON string
json_string = expanded_df.to_json(orient='records')

# Get the bucket and file objects
bucket = client.get_bucket('hons-project-datasets')
blob = bucket.blob('modified/train_skip_no_refs_expanded_cleaned.json')

# Upload the JSON string to the bucket
blob.upload_from_string(json_string, content_type='application/json')

In [ ]:
expanded_df

,example_id,title,unit_of_measure,chart_type,was_translated,table_data,table_text,linearized_input
0,AB20-ar-1,تمكين الشباب بين الشابات في الفئة العمرية من 1...,النسبة المئوية موضحة في ثلاثة أجزاء مُجمعة,Horizontal Bar Chart,True,"[["""", ""\u0645\u0633\u062a\u0648\u0649 \u0639\u...",تتمتع 13% من الشابات في مالي بأعلى مستويات الت...,تمكين الشباب بين الشابات في الفئة العمرية من 1...
0,AB20-ar-1,تمكين الشباب بين الشابات في الفئة العمرية من 1...,النسبة المئوية موضحة في ثلاثة أجزاء مُجمعة,Horizontal Bar Chart,True,"[["""", ""\u0645\u0633\u062a\u0648\u0649 \u0639\u...",تحتل نيجيريا منتصف المجموعات، حيث تتضمن كل مجم...,تمكين الشباب بين الشابات في الفئة العمرية من 1...
1,AB20-ar-2,العدد المثالي للأطفال وفقًا لمستوى التمكين - ن...,العدد بين الشابات في الفئة العمرية من 15 إلى 2...,Line Chart,True,"[[""\u0645\u0633\u062a\u0648\u0649 \u0645\u0646...",في نيجيريا، ترغب الشابات ذوات المستوى المنخفض ...,العدد المثالي للأطفال وفقًا لمستوى التمكين - ن...
2,AB20-ar-3,العدد المثالي للأطفال وتمكين الشابات: تحليل مت...,معاملات تمكين الشابات للعدد المثالي للأطفال من...,Other,True,"[["""", ""\u0645\u0633\u062a\u0648\u0649 \u0645\u...",تُظهر النماذج متعددة المتغيرات أنه في 5 بلدان،...,العدد المثالي للأطفال وتمكين الشابات: تحليل مت...
2,AB20-ar-3,العدد المثالي للأطفال وتمكين الشابات: تحليل مت...,معاملات تمكين الشابات للعدد المثالي للأطفال من...,Other,True,"[["""", ""\u0645\u0633\u062a\u0648\u0649 \u0645\u...",حتى عند التحكم في الخصائص الخلفية، ترغب الشابا...,العدد المثالي للأطفال وتمكين الشابات: تحليل مت...
...,...,...,...,...,...,...,...,...
6959,WP120-sw-2,Asilimia ya wanawake waliotumia aina tofauti z...,CPR(%),Vertical Bar Chart,True,"[[""Sindano"", 5.7], [""Kondomu ya mwanaume"", 3.4...",Kielelezo 4 kinaonyesha mseto wa njia za kisas...,Asilimia ya wanawake waliotumia aina tofauti z...
6959,WP120-sw-2,Asilimia ya wanawake waliotumia aina tofauti z...,CPR(%),Vertical Bar Chart,True,"[[""Sindano"", 5.7], [""Kondomu ya mwanaume"", 3.4...","Utumiaji wa njia za kudumu, hasa ufanyaji wa w...",Asilimia ya wanawake waliotumia aina tofauti z...
6960,WP120-yo-1,Ìtànkálẹ̀ àwọn ohun èèlò ìfètòsọmọbi...,Òdiwọ̀n ìdá nínú ìdá ọgọ́ọ̀rún,Vertical Bar Chart,True,"[[""O\u0323\u0300na\u0300 a\u0300tis\u0323e e\u...","Ní àpapọ̀, ìdá 26 nínú ìdá ọgọ́rùn à...",Ìtànkálẹ̀ àwọn ohun èèlò ìfètòsọmọbi...
6961,WP120-yo-2,Ìdá nínú ìdá ọgọ́rùn àwọn obìnrin ló...,CPR (Ìdá nínú ìdá ọgọ́rùn),Vertical Bar Chart,True,"[[""A\u0300wo\u0323n to\u0301s\u0323e\u0301egu\...",Àwòrán 4 ṣàfihàn àlòpapọ̀ àwọn ohun è...,Ìdá nínú ìdá ọgọ́rùn àwọn obìnrin ló...
